# 🔗 Data Merging Notebook – Milestone 2

**Project:** Gold Pathfinder ML Project  
**Notebook:** `02_data_merging.ipynb`  

This notebook documents how the **cleaned per-type datasets** are combined
into a **single unified dataset**:

```text
1_datasets/processed/gold_assays_final.csv
```

It mirrors the merge logic in:

```text
2_data_preparation/scripts/data_preparation.py
```

and serves as a transparent, visual version for ELO2 evaluation.


In [ ]:
from pathlib import Path
import pandas as pd

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

PROJECT_ROOT = Path('..').resolve()
CLEANED_DIR = PROJECT_ROOT / '1_datasets' / 'cleaned'
PROCESSED_DIR = PROJECT_ROOT / '1_datasets' / 'processed'

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

PROJECT_ROOT, CLEANED_DIR, PROCESSED_DIR

## 1️⃣ Load Cleaned Per-Type Datasets

We expect the following files in `1_datasets/cleaned/`:

- `core_assays_clean.csv`
- `rc_assays_clean.csv`
- `chip_assays_clean.csv`
- `trench_assays_clean.csv`
- `grab_assays_clean.csv`


In [ ]:
files = {
    'core': CLEANED_DIR / 'core_assays_clean.csv',
    'rc': CLEANED_DIR / 'rc_assays_clean.csv',
    'chip': CLEANED_DIR / 'chip_assays_clean.csv',
    'trench': CLEANED_DIR / 'trench_assays_clean.csv',
    'grab': CLEANED_DIR / 'grab_assays_clean.csv',
}

for name, path in files.items():
    print(name, '->', path, 'exists:', path.exists())

In [ ]:
dfs = {}
for name, path in files.items():
    if path.exists():
        df = pd.read_csv(path)
        df['sample_type'] = df.get('sample_type', name)
        dfs[name] = df
        print(f'Loaded {name}: {df.shape[0]} rows, {df.shape[1]} columns')
    else:
        print(f'WARNING: {name} file is missing, skipping.')

## 2️⃣ Harmonize Columns and Merge

We build the union of all columns, reindex each dataframe, and then
concatenate them into a single table.


In [ ]:
if not dfs:
    raise RuntimeError('No cleaned datasets loaded. Please check CLEANED_DIR.')

all_cols = sorted(set().union(*[df.columns for df in dfs.values()]))
all_cols

In [ ]:
aligned_dfs = [df.reindex(columns=all_cols) for df in dfs.values()]
final_df = pd.concat(aligned_dfs, ignore_index=True)
final_df.shape

Quick preview of the final merged dataset:

In [ ]:
final_df.head()

## 3️⃣ Basic Quality Checks

- Missing `sample_id` values
- Presence of `sample_type`
- Range of `au_ppm` if available


In [ ]:
final_df['sample_id'].isna().sum(), final_df['sample_type'].isna().sum()

In [ ]:
if 'au_ppm' in final_df.columns:
    print('Min Au:', final_df['au_ppm'].min())
    print('Max Au:', final_df['au_ppm'].max())
else:
    print("WARNING: 'au_ppm' column not found in final_df")

## 4️⃣ Save Final Dataset

We now save the merged dataset to:

```text
1_datasets/processed/gold_assays_final.csv
```


In [ ]:
final_out = PROCESSED_DIR / 'gold_assays_final.csv'
final_df.to_csv(final_out, index=False)
final_out

## ✅ Milestone 2 Output

At this point, we have:

- Cleaned per-type datasets in `1_datasets/cleaned/`
- A unified merged dataset in `1_datasets/processed/gold_assays_final.csv`

This completes the **Data Collection & Preparation (Milestone 2)** stage.
Next, in Milestone 3, we will perform **exploratory data analysis (EDA)**
and start identifying candidate **geochemical pathfinder elements**.
